# DRAG 脉冲
*版权所有 (c) 2021 百度量子计算研究所，保留所有权利。*

## 内容概要
本教程将演示如何使用量脉实现一个采用 DRAG（Derivative Reduction by Adiabatic Gate）方案生成脉冲实现的 X 门，并且与传统的 $\pi$ 脉冲方案进行对比，本教程的大纲如下：

- 背景介绍
- 准备工作
- 定义 DRAG 波形
- 量脉实现
- 总结

## 背景介绍

由于超导电路不是理想的二能级系统，因此需要考虑能级泄漏引入的误差。对于弱失谐量子比特，能量泄漏到第三能级会使量子比特的状态脱离计算空间。为了克服这个问题，研究人员提出了 DRAG 方案 \[1\]，它通过修正驱动脉冲的波形来消除能级泄露引入的误差。

## 准备工作
成功安装量脉后，可以按照本教程运行下面的量脉程序。要运行此教程，您需要从 Quanlse 和其它常用的 Python 库导入以下包：

In [ ]:
# Import the Hamiltonian module
from Quanlse.QHamiltonian import QHamiltonian as QHam

# Import the function for calculating infidelity
from Quanlse.Utils.Infidelity import unitaryInfidelity

# Import related operators
from Quanlse.QOperator import driveX, driveY, driveZ, number, duff

# Import waveforms and functions used to process the waveforms' data
from Quanlse.QWaveform import gaussian, dragY1, QJob, QJobList, QWaveform

# Import simulator interface for Quanlse Cloud Service
from Quanlse.remoteSimulator import remoteSimulatorRunHamiltonian as runHamiltonian

# Import matplotlib for graphing purposes
import matplotlib.pyplot as plt

# Import numpy and scipy
import numpy as np
import math
from scipy import integrate

为了使用量脉云服务，我们需要获得一个 token 接入云端。

In [ ]:
# Import Define class and set the token for cloud service
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ""

## 定义 DRAG 波形

我们考虑一个 transmon 量子比特，通常通过向 XY 通道施加微波脉冲实现高保真度的 X 门。将量子比特第一激发态的能量和第二激发态能量分别记为 $\omega_1$ 和 $\omega_2$，驱动频率记为 $\omega_d$。利用旋转波近似，该系统的哈密顿量可以写成 \[2\]：

$$ 
\hat H_R / \hbar = \delta_1 |1\rangle \langle 1|+\delta_2 |2\rangle \langle 2|+\frac{\alpha_q}{2}\hat a^{\dagger}\hat a^{\dagger}\hat a \hat a+\frac{\varepsilon_x(t)}{2}
\left[ \hat{a}^\dagger + \hat{a} \right]+\frac{\varepsilon_y(t)}{2}
i \left[\hat{a}^\dagger - \hat{a}\right]
,
$$

其中 $\alpha_q = \omega_2 -2\omega_1$ 是系统的失谐强度。$\delta_1 = \omega_1-\omega_d$ 和 $\delta_2 = \omega_2-\omega_d$ 是量子比特跃迁频率相对于驱动频率的失调。此外 $\varepsilon_x(t)$ 和 $\varepsilon_y(t)$ 是施加到 XY 通道的脉冲函数。

在理想情况下，我们可以忽略 transmon 量子比特的更高能级。我们通常会施加与量子比特频率相等的脉冲，即 $\delta _1$ 设为零。欲想得到该 X 通道的脉冲，我们直接求解方程：
$$
\int_0^{t_g}\varepsilon_x(t)dt=\theta. 
$$

对于高斯波形 $\varepsilon_G=Ae^{(t-\tau)^2 /2\sigma^2}-B$，上述待求解方程变为 $\int_0^{t_g}\varepsilon_G(t)dt=\theta$，于是得到绕 x 轴转动 $\theta$ 所对应的脉冲振幅 $A$：
$$
A=\theta/\left( \int_0^{t_g}e^{-(t-\tau)^2/2\sigma^2}dt-t_ge^{-\tau^2/2\sigma^2} \right),
$$

$$
B=Ae^{-\tau^2/2\sigma^2}.
$$
在上面的等式中，$A$ 表示实现旋转量子门所需的脉冲振幅；而 $B$ 使得在开始时刻和结束时刻脉冲的振幅为零。

在下面的代码中，我们首先设置系统的在布洛赫球上的旋转角度以及失谐性。然后，我们定义计算高斯波形脉冲值的函数（量脉提供了常用波形的函数）。

In [ ]:
theta_x = np.pi # The angle of rotation
Delta = -0.4 * 2 * np.pi # The anharmonicity in GHz

# Calculate the parameters
def intTheta(tg):
    y = integrate.quad(gaussian(0, tg, 1, tg / 2, tg / 4), 0, tg)
    return y[0]

def calAx(tg):
    return theta_x / (intTheta(tg) - gaussian(0, tg, 1, tg / 2, tg / 4)(0) * tg)

def calBx(tg):
    return calAx(tg) * gaussian(0, tg, 1, tg / 2, tg / 4)(0)

DRAG 脉冲的波形和失谐分别为：
$$
\varepsilon_y(t) = -\frac{\dot {\varepsilon_x}(t)}{\alpha_q}, 
$$
$$
\delta_1(t) = -\frac{\varepsilon_x^2(t)}{2\alpha_q}.
$$

这里，我们根据上面的方程建立控制脉冲 $\varepsilon_x(t)$ 和 $\varepsilon_y(t)$，并设置驱动脉冲的失谐 $\delta_1$：

In [ ]:
# Define the control waveforms
def epsilonX(t, params):
    tg = params['tg']
    a = calAx(tg)
    b = calBx(tg)
    return gaussian(0, tg, a, tg / 2, tg / 4)(t) - b
    
def epsilonY(t, params):
    tg = params['tg']
    a = calAx(tg)
    return dragY1(0, tg, a, tg / 2, tg / 4)(t) / Delta

# Set the drive detuning  
def delta1(t, params):
    tg = params['tg']
    lamda = np.sqrt(2)
    return - epsilonX(t, {"tg": tg}) ** 2 / 2 / Delta

## 量脉实现

量脉将模拟和优化所需的系统信息存储在哈密顿量的字典中。首先，我们通过实例化一个 `QHamiltonian` 的对象 'ham' 创建一个空的哈密顿量。为了做一个对比，我们创建了两个空的哈密顿量，'ham' 不使用DRAG方法，'hamDrag' 将会使用DRAG方法。

In [ ]:
# Create empty Hamiltonians
ham = QHam(subSysNum=1, sysLevel=3, dt=0.2222)
hamDrag = QHam(subSysNum=1, sysLevel=3, dt=0.2222)

对于这一特定任务，系统哈密顿量可分为四个部分：
$$
\hat H_R = \hat H_{\rm drift} + \hat H_{\rm xctrl} + \hat H_{\rm yctrl}+ \hat H_{\rm freq} ,
$$
其中 $\hat H_{\rm drift}= \alpha_q\hat a^{\dagger}\hat a^{\dagger}\hat a \hat a \, / \, 2$ 表示量子比特的失谐性，是超导量子比特的内在特性，且与时间无关，因而我们可以通过调用 `addDrift()` 来添加该项。算符 $\hat{a}^{\dagger}\hat{a}^{\dagger} \hat{a} \hat{a}$ 在量脉中用 `duff()` 来表示，它将系统的能级数量作为输入参数。

In [ ]:
# Add the anharmonic terms
ham.addDrift(duff(3), onSubSys=0, coef=Delta / 2.0)
hamDrag.addDrift(duff(3), onSubSys=0, coef=Delta / 2.0)

下一步，通过调用 `addWave()` 添加控制项 $\hat H_{\rm xctrl}=(\hat a +\hat a^{\dagger})/2$、$\hat H_{\rm yctrl}=i(\hat a -\hat a^{\dagger})/2$ 和 $ \hat H_{\rm freq}=\hat a^{\dagger}\hat a $。我们可以在 `QOperator` 中找到相应的算符。这里我们创建一个 `QJobList` 的对象 'JobList ' 来统一添加控制波形。代码如下:

In [ ]:
# Gate times
t = np.arange(2., 9., 0.5)

# Intialize array index
jobList = ham.createJobList()
jobListDrag = hamDrag.createJobList()
for tg in t:
    jobWaves = jobList.createJob()
    jobWavesDrag = jobListDrag.createJob()
    # Add Gaussian Wave of X control on the qubit 0
    paraArgs = {"a": -0.5 * 2.0 * np.pi}
    # Add wave for the job list without DRAG pulses
    jobWaves.addWave(driveX(3), 0, QWaveform(epsilonX, 0, tg, {"tg": tg}))
    # Add wave for the job list with DRAG pulses
    jobWavesDrag.addWave(driveX(3), 0, QWaveform(epsilonX, 0, tg, {"tg": tg}))
    jobWavesDrag.addWave(driveY(3), 0, QWaveform(epsilonY, 0, tg, {"tg": tg}))
    jobWavesDrag.addWave(driveZ(3), 0, QWaveform(delta1, 0, tg, {"tg": tg}))
    # Append this job to the job list
    jobList.addJob(jobWaves)
    jobListDrag.addJob(jobWavesDrag)

为了对 DRAG 脉冲方案和传统的 $\pi$ 脉冲方案进行完整的对比，我们在不同的门时间下计算量子门的保真度。使用量脉可以非常高效地完成这一任务，我们提供的函数 `runHamiltonian()` 允许用户向云端提交批量任务。它返回一个包含详细结果的字典列表，酉矩阵存储在每个字典的键 `"unitary"` 下。

在本地设备上进行模拟可能需要很长时间，量脉提供的云服务可以显著加快这一过程。要使用量脉云服务，用户可以在百度量易伏网站 [http://quantum-hub.baidu.com](http://quantum-hub.baidu.com) 上获取 token，然后使用 `remoteSimulator()` 模块中的函数将任务提交到量脉云服务上。

下一步是计算我们刚刚获得的实际酉矩阵和理想 X 门之间的距离。量脉提供了 `unitaryInfidelity()` 的函数，该函数能够根据下式计算失真度：
$$
{\rm infid} =1- \frac{1}{2}\left|{\rm Tr}(\hat{\sigma}_x P(U))\right|.
$$
其中投影演化 $P(U)$（$U$是系统的演化算符）是描述投影到由最低的两个能量本征态 $|0\rangle$ 和 $|1\rangle$ 构成的计算空间的演化；$\hat{\sigma}_x$ 是我们想要实现的目标门。

In [ ]:
# Create the arrays for storing gate infidelities
errorX = np.zeros(len(t))
errorXDrag = np.zeros(len(t))

# Submit the job lists to Quanlse Cloud Service
result = runHamiltonian(ham, jobList=jobList)
resultDrag = runHamiltonian(hamDrag, jobList=jobListDrag)
errorX = []
errorXDrag = []

for index in range(len(t)):
    errorX.append(unitaryInfidelity(np.array([[0, 1], [1, 0]], dtype=complex), result[index]["unitary"], 1))
    errorXDrag.append(unitaryInfidelity(np.array([[0, 1], [1, 0]], dtype=complex), resultDrag[index]["unitary"], 1))

最后，我们可以使用 Matplotlib 库对结果进行分析和可视化。

In [ ]:
plt.semilogy(t, errorXDrag, label='With DRAG', marker='.')
plt.semilogy(t, errorX, label='Without DRAG', marker='.')

plt.xlabel('Gate Time (ns)')
plt.ylabel('Infidelity')
plt.title('X Gate')
plt.legend()
plt.show()

如上所示，我们消除了大部分的能级泄露误差。蓝色（DRAG 优化）曲线显示 DRAG 脉冲显著地提升了 X 门的保真度。

## 总结
本教程使用量脉呈现了 DRAG 的优化方案与传统的 $\pi$ 脉冲方案之间的对比。我们鼓励用户尝试不同于本教程的参数值以获得最佳结果。

## 参考文献

\[1\] [Motzoi, Felix, et al. "Simple pulses for elimination of leakage in weakly nonlinear qubits." *Physical review letters* 103.11 (2009): 110501.](https://link.aps.org/doi/10.1103/PhysRevLett.103.110501)

\[2\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://aip.scitation.org/doi/abs/10.1063/1.5089550)